In [1]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import XSD, RDF
from datetime import datetime

# Create a namespace for humemai
humemai = Namespace("https://humem.ai/ontology/")

# Initialize the RDF graph
graph = Graph()
graph.bind("humemai", humemai)

# Define some sample triples with currentTime
triple1 = (
    URIRef("https://example.org/person/Alice"),
    humemai.currentTime,
    Literal("2024-04-27T10:00:00", datatype=XSD.dateTime),
)

triple2 = (
    URIRef("https://example.org/person/Charlie"),
    humemai.currentTime,
    Literal(datetime.now().isoformat(), datatype=XSD.dateTime),
)

# Add the triples to the graph
graph.add(triple1)
graph.add(triple2)

# Display all triples in the graph
for subj, pred, obj in graph:
    print(f"Triple: ({subj}, {pred}, {obj})")

# Use graph.objects() to iterate over all currentTime values
print("\nRetrieved currentTime values:")
for obj in graph.objects(None, humemai.currentTime):
    print(f"currentTime value: {obj}")

# Alternative: If you want to retrieve all triples with currentTime:
print("\nAll triples with currentTime predicate:")
for subj, obj in graph.subject_objects(humemai.currentTime):
    print(f"Subject: {subj}, currentTime: {obj}")


Triple: (https://example.org/person/Charlie, https://humem.ai/ontology/currentTime, 2024-10-04T12:30:06.141966)
Triple: (https://example.org/person/Alice, https://humem.ai/ontology/currentTime, 2024-04-27T10:00:00)

Retrieved currentTime values:
currentTime value: 2024-04-27T10:00:00
currentTime value: 2024-10-04T12:30:06.141966

All triples with currentTime predicate:
Subject: https://example.org/person/Alice, currentTime: 2024-04-27T10:00:00
Subject: https://example.org/person/Charlie, currentTime: 2024-10-04T12:30:06.141966


In [2]:
graph.value(URIRef("https://example.org/person/Alice"), humemai.currentTime)

rdflib.term.Literal('2024-04-27T10:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime'))

In [3]:
for s, p, o in graph:
    print(s, p, o)

https://example.org/person/Charlie https://humem.ai/ontology/currentTime 2024-10-04T12:30:06.141966
https://example.org/person/Alice https://humem.ai/ontology/currentTime 2024-04-27T10:00:00


In [4]:
import logging
from rdflib import URIRef, Namespace
from humemai import MemorySystem
from rdflib import BNode, Graph, Literal, Namespace, URIRef

# Define the custom namespace for humemai ontology
humemai = Namespace("https://humem.ai/ontology/")

# Turn off debug logging
logging.getLogger("humemai").setLevel(logging.WARNING)

# Initialize the memory system
memory_system = MemorySystem()
memory_system.memory.add_memory(
    [(humemai.foo, humemai.bar, humemai.baz), (humemai.foo, humemai.bar, humemai.baz)],
    {"foo": "bar"},
)
memory_system

ValueError: Qualifier key foo must be a URIRef.

In [69]:
RDF.subject

rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#subject')

In [73]:
from datetime import datetime

foo = Literal(datetime.now().isoformat(), datatype=XSD.dateTime)

In [76]:
Literal("dd")

rdflib.term.Literal('dd')

In [ ]:
memory_system.memory.get_memories(event="CP000")

[P4069, hasSituation, S00, {'memoryID': '0', 'location': 'USAR', 'time': '2024-05-03T13:41:23', 'event': 'CP000', 'recalled': '0'}]

In [ ]:
memory_system.memory.get_memories(event="CP000")

[P4069, hasSituation, S00, {'memoryID': '0', 'location': 'USAR', 'time': '2024-05-03T13:41:23', 'event': 'CP000', 'recalled': '0'}]

In [ ]:
memory_system.memory.get_memories(event="CP000")

[P4069, hasSituation, S00, {'memoryID': '0', 'location': 'USAR', 'time': '2024-05-03T13:41:23', 'event': 'CP000', 'recalled': '0'}]

In [75]:
str(foo)

'2024-10-04T10:56:54.117753'

In [72]:
foo.n3()

'"2024-10-04T10:37:02.644971"^^<http://www.w3.org/2001/XMLSchema#dateTime>'

In [68]:
foo.datatype == XSD.dateTime

False

In [62]:
foo is aiterf

rdflib.term.Literal('2024-10-04T10:13:43.281562', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime'))

In [53]:
URIRef("https://humem.ai/ontology/currentTime")

rdflib.term.URIRef('https://humem.ai/ontology/currentTime')

In [54]:
humemai.currentTime in {URIRef("https://humem.ai/ontology/currentTime"):1}

True

In [50]:
Literal("bar")

rdflib.term.Literal('bar')

In [46]:
def foo(bar: Literal) -> Literal:
    return bar

foo

{rdflib.term.Literal('bar'): 1}

In [47]:
event_time = "2023-10-01T10:00:00"

Literal(event_time, datatype=XSD.dateTime)

rdflib.term.Literal('2023-10-01T10:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime'))

In [48]:
humemai.Event

rdflib.term.URIRef('https://humem.ai/ontology/Event')

In [26]:
from rdflib.namespace import RDF, XSD

In [30]:
RDF

Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')

In [28]:
XSD

/home/tk/.virtualenvs/humemai/lib/python3.10/site-packages/IPython/utils/dir2.py:65: UserWarning: Code: _ipython_canary_method_should_not_exist_ is not defined in namespace XSD
  canary = getattr(obj, '_ipython_canary_method_should_not_exist_', None)


Namespace('http://www.w3.org/2001/XMLSchema#')

In [25]:
URIRef(URIRef("https://humem.ai/ontology/foo"))

rdflib.term.URIRef('https://humem.ai/ontology/foo')

In [23]:
g = Graph()

g.add(("a", "b", "c"))

AssertionError: Subject a must be an rdflib term

In [20]:
{"a": 1, **{humemai.currentTime: 1}}

{'a': 1, rdflib.term.URIRef('https://humem.ai/ontology/currentTime'): 1}

In [17]:
from datetime import datetime


datetime.fromisoformat("2024-04-27T15:00:00")

datetime.datetime(2024, 4, 27, 15, 0)

In [14]:
foo = {"d": 1}.get(1)
foo == None

True

In [9]:
if {"a": 1}:
    print('asdf')

asdf


In [7]:
humemai.Foo

rdflib.term.URIRef('https://humem.ai/ontology/Foo')

In [22]:
Literal(1)

rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer'))

In [17]:
memory_system.memory.get_triple_count_except_event()

1

In [5]:
my_dict = {}
{"a": "b", **my_dict}

{'a': 'b'}

In [1]:
import logging
from rdflib import URIRef, Namespace
from humemai import MemorySystem

# Define the custom namespace for humemai ontology
humemai = Namespace("https://humem.ai/ontology/")

# Turn off debug logging
logging.getLogger("humemai").setLevel(logging.WARNING)

# Initialize the memory system
memory_system = MemorySystem()

# Define multiple triples
triples = [
    (
        URIRef("https://example.org/person/Alice"),
        URIRef("https://example.org/event/met"),
        URIRef("https://example.org/person/Bob"),
    ),  # 0
    (
        URIRef("https://example.org/person/Bob"),
        URIRef("https://example.org/event/met"),
        URIRef("https://example.org/person/Charlie"),
    ),  # 1
    (
        URIRef("https://example.org/person/Alice"),
        URIRef("https://example.org/relationship/knows"),
        URIRef("https://example.org/person/David"),
    ),  # 2
    (
        URIRef("https://example.org/person/Charlie"),
        URIRef("https://example.org/event/met"),
        URIRef("https://example.org/person/Eve"),
    ),  # 3
    (
        URIRef("https://example.org/person/David"),
        URIRef("https://example.org/event/invited"),
        URIRef("https://example.org/person/Alice"),
    ),  # 4
    (
        URIRef("https://example.org/person/Eve"),
        URIRef("https://example.org/relationship/worksWith"),
        URIRef("https://example.org/person/Bob"),
    ),  # 5
    (
        URIRef("https://example.org/person/Charlie"),
        URIRef("https://example.org/event/attended"),
        URIRef("https://example.org/person/Bob"),
    ),  # 6
    (
        URIRef("https://example.org/entity/Dog"),
        URIRef("https://example.org/relationship/is"),
        URIRef("https://example.org/entity/Animal"),
    ),  # 7
    (
        URIRef("https://example.org/person/Alice"),
        URIRef("https://example.org/relationship/owns"),
        URIRef("https://example.org/entity/Dog"),
    ),  # 8
    (
        URIRef("https://example.org/person/Eve"),
        URIRef("https://example.org/event/met"),
        URIRef("https://example.org/person/Alice"),
    ),  # 9
    (
        URIRef("https://example.org/person/John"),
        URIRef("https://example.org/relationship/loves"),
        URIRef("https://example.org/entity/Animal"),
    ),  # 10
]

# Define qualifiers for episodic memories
episodic_qualifiers_1 = {
    "location": "New York",
    "time": "2024-04-27T15:00:00",
    "emotion": "happy",
    "event": "Coffee meeting",
}
episodic_qualifiers_2 = {
    "location": "London",
    "time": "2024-05-01T10:00:00",
    "emotion": "excited",
    "event": "Conference meeting",
}
episodic_qualifiers_3 = {
    "location": "Paris",
    "time": "2024-05-03T14:00:00",
    "emotion": "curious",
    "event": "Workshop",
}

# Add long-term episodic memories
memory_system.memory.add_long_term_memory(
    "episodic", [triples[0]], **episodic_qualifiers_1
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[1]], **episodic_qualifiers_2
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[2]], **episodic_qualifiers_3
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[3]], **episodic_qualifiers_1
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[4]], **episodic_qualifiers_2
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[5]], **episodic_qualifiers_3
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[6]], **episodic_qualifiers_1
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[7]], **episodic_qualifiers_2
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[8]], **episodic_qualifiers_3
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[9]], **episodic_qualifiers_1
)
memory_system.memory.add_long_term_memory(
    "episodic", [triples[10]], **episodic_qualifiers_2
)


# Add a semantic memory for the "Dog" entity
semantic_qualifiers = {"derivedFrom": "animal_research", "strength": 5}
memory_system.memory.add_long_term_memory(
    "semantic", [triples[7]], **semantic_qualifiers
)
memory_system.memory.add_long_term_memory(
    "semantic", [triples[7]], **semantic_qualifiers
)
memory_system.memory.add_long_term_memory(
    "semantic", [triples[10]], **semantic_qualifiers
)
memory_system.memory.add_long_term_memory(
    "semantic", [triples[10]], **semantic_qualifiers
)

# Add a short-term memory (recent event)
memory_system.memory.add_short_term_memory([triples[8]], location="Alice's home")
memory_system.memory.add_short_term_memory([triples[8]], location="Alice's home")

# Add another short-term memory
memory_system.memory.add_short_term_memory([triples[9]], location="Paris Cafe")
memory_system.memory.add_short_term_memory([triples[9]], location="Paris Cafe")


# Function to count triples and memories in working memory
def print_memory_statistics(working_memory, hop_level):
    num_triples = working_memory.get_triple_count_except_event()
    num_memories = working_memory.get_memory_count()  # This counts reified statements

    print(f"\n===== Working Memory (Hop {hop_level}) =====")
    print(f"Number of Triples: {num_triples}")
    print(f"Number of Memories (Reified Statements): {num_memories}")
    print(repr(working_memory))


# Test working memory retrieval with different hops
trigger_node = URIRef("https://example.org/person/Alice")

# Hop 0 - Just the immediate node (only the short-term memories involving Alice)
working_memory_hop_0 = memory_system.get_working_memory(
    trigger_node=trigger_node, hops=0
)
print_memory_statistics(working_memory_hop_0, 0)

# Hop 1 - Immediate neighbors (both incoming and outgoing edges)
working_memory_hop_1 = memory_system.get_working_memory(
    trigger_node=trigger_node, hops=1
)
print_memory_statistics(working_memory_hop_1, 1)

# Hop 2 - 2-hop neighbors (further expansion of relationships)
working_memory_hop_2 = memory_system.get_working_memory(
    trigger_node=trigger_node, hops=2
)
print_memory_statistics(working_memory_hop_2, 2)

# Hop 3 - 3-hop neighbors (going even further in the memory graph)
working_memory_hop_3 = memory_system.get_working_memory(
    trigger_node=trigger_node, hops=3
)
print_memory_statistics(working_memory_hop_3, 3)

# Hop 4 - Maximum depth
working_memory_hop_4 = memory_system.get_working_memory(
    trigger_node=trigger_node, hops=4
)
print_memory_statistics(working_memory_hop_4, 4)

# Hop 5 - Maximum depth
working_memory_hop_5 = memory_system.get_working_memory(
    trigger_node=trigger_node, hops=5
)
print_memory_statistics(working_memory_hop_5, 5)

# Example: Get working memory with all long-term memories included
working_memory_all = memory_system.get_working_memory(include_all_long_term=True)
print_memory_statistics(working_memory_all, "All long-term")

AttributeError: 'Memory' object has no attribute 'add_long_term_memory'

In [20]:
[triples[0]]

[(rdflib.term.URIRef('https://example.org/person/Alice'),
  rdflib.term.URIRef('https://example.org/event/met'),
  rdflib.term.URIRef('https://example.org/person/Bob'))]

In [2]:
memory_system

[Alice, met, Bob, {'memoryID': '0', 'location': 'New York', 'time': '2024-04-27T15:00:00', 'emotion': 'happy', 'event': 'Coffee meeting', 'recalled': '6'}]
[Bob, met, Charlie, {'memoryID': '1', 'location': 'London', 'time': '2024-05-01T10:00:00', 'emotion': 'excited', 'event': 'Conference meeting', 'recalled': '5'}]
[Alice, knows, David, {'memoryID': '2', 'location': 'Paris', 'time': '2024-05-03T14:00:00', 'emotion': 'curious', 'event': 'Workshop', 'recalled': '6'}]
[Charlie, met, Eve, {'memoryID': '3', 'location': 'New York', 'time': '2024-04-27T15:00:00', 'emotion': 'happy', 'event': 'Coffee meeting', 'recalled': '5'}]
[David, invited, Alice, {'memoryID': '4', 'location': 'London', 'time': '2024-05-01T10:00:00', 'emotion': 'excited', 'event': 'Conference meeting', 'recalled': '6'}]
[Eve, worksWith, Bob, {'memoryID': '5', 'location': 'Paris', 'time': '2024-05-03T14:00:00', 'emotion': 'curious', 'event': 'Workshop', 'recalled': '5'}]
[Charlie, attended, Bob, {'memoryID': '6', 'location

In [3]:
memory_system.memory.save_to_ttl("foo.ttl")

In [4]:
import logging
from rdflib import URIRef, Namespace
from humemai import MemorySystem

# Define the custom namespace for humemai ontology
humemai = Namespace("https://humem.ai/ontology/")

# Turn off debug logging
logging.getLogger("humemai").setLevel(logging.WARNING)

# Initialize the memory system
memory_system = MemorySystem()

memory_system.memory.load_from_ttl("foo.ttl")

memory_system

[Alice, owns, Dog, {'emotion': 'curious', 'event': 'Workshop', 'location': 'Paris', 'memoryID': '8', 'recalled': '6', 'time': '2024-05-03T14:00:00'}]
[Dog, is, Animal, {'derivedFrom': 'animal_research', 'memoryID': '11', 'recalled': '5', 'strength': '5'}]
[Alice, met, Bob, {'emotion': 'happy', 'event': 'Coffee meeting', 'location': 'New York', 'memoryID': '0', 'recalled': '6', 'time': '2024-04-27T15:00:00'}]
[Eve, met, Alice, {'currentTime': '2024-10-03T21:49:07.791818', 'location': 'Paris Cafe', 'memoryID': '18'}]
[Alice, knows, David, {'emotion': 'curious', 'event': 'Workshop', 'location': 'Paris', 'memoryID': '2', 'recalled': '6', 'time': '2024-05-03T14:00:00'}]
[Alice, owns, Dog, {'currentTime': '2024-10-03T21:49:07.791643', 'location': "Alice's home", 'memoryID': '16'}]
[John, loves, Animal, {'emotion': 'excited', 'event': 'Conference meeting', 'location': 'London', 'memoryID': '10', 'recalled': '4', 'time': '2024-05-01T10:00:00'}]
[Bob, met, Charlie, {'emotion': 'excited', 'event

In [5]:
memory_system.memory

[Alice, owns, Dog, {'emotion': 'curious', 'event': 'Workshop', 'location': 'Paris', 'memoryID': '8', 'recalled': '6', 'time': '2024-05-03T14:00:00'}]
[Dog, is, Animal, {'derivedFrom': 'animal_research', 'memoryID': '11', 'recalled': '5', 'strength': '5'}]
[Alice, met, Bob, {'emotion': 'happy', 'event': 'Coffee meeting', 'location': 'New York', 'memoryID': '0', 'recalled': '6', 'time': '2024-04-27T15:00:00'}]
[Eve, met, Alice, {'currentTime': '2024-10-03T21:49:07.791818', 'location': 'Paris Cafe', 'memoryID': '18'}]
[Alice, knows, David, {'emotion': 'curious', 'event': 'Workshop', 'location': 'Paris', 'memoryID': '2', 'recalled': '6', 'time': '2024-05-03T14:00:00'}]
[Alice, owns, Dog, {'currentTime': '2024-10-03T21:49:07.791643', 'location': "Alice's home", 'memoryID': '16'}]
[John, loves, Animal, {'emotion': 'excited', 'event': 'Conference meeting', 'location': 'London', 'memoryID': '10', 'recalled': '4', 'time': '2024-05-01T10:00:00'}]
[Bob, met, Charlie, {'emotion': 'excited', 'event

In [6]:
memory_system.memory.get_triple_count_except_event()

11

In [7]:
memory_system.memory.get_memory_count()

19

In [8]:
memory_system.memory.get_short_term_memory_count()

4

In [9]:
memory_system.memory.get_long_term_memory_count()

15

In [10]:
memory_system.memory.get_long_term_episodic_memory_count()

11

In [11]:
memory_system.memory.get_long_term_semantic_memory_count()

4

In [12]:
for memory in memory_system.memory.iterate_memories("short_term"):
    print(memory)

(rdflib.term.URIRef('https://example.org/person/Eve'), rdflib.term.URIRef('https://example.org/event/met'), rdflib.term.URIRef('https://example.org/person/Alice'), {'https://humem.ai/ontology/currentTime': '2024-10-03T21:49:07.791818', 'https://humem.ai/ontology/location': 'Paris Cafe', 'https://humem.ai/ontology/memoryID': '18'})
(rdflib.term.URIRef('https://example.org/person/Alice'), rdflib.term.URIRef('https://example.org/relationship/owns'), rdflib.term.URIRef('https://example.org/entity/Dog'), {'https://humem.ai/ontology/currentTime': '2024-10-03T21:49:07.791643', 'https://humem.ai/ontology/location': "Alice's home", 'https://humem.ai/ontology/memoryID': '16'})
(rdflib.term.URIRef('https://example.org/person/Alice'), rdflib.term.URIRef('https://example.org/relationship/owns'), rdflib.term.URIRef('https://example.org/entity/Dog'), {'https://humem.ai/ontology/currentTime': '2024-10-03T21:49:07.791558', 'https://humem.ai/ontology/location': "Alice's home", 'https://humem.ai/ontology

In [13]:
len(list(memory_system.memory.iterate_memories("long_term")))

15

In [14]:
for memory in memory_system.memory.iterate_memories("long_term"):
    print(memory)

(rdflib.term.URIRef('https://example.org/person/Alice'), rdflib.term.URIRef('https://example.org/relationship/owns'), rdflib.term.URIRef('https://example.org/entity/Dog'), {'https://humem.ai/ontology/emotion': 'curious', 'https://humem.ai/ontology/event': 'Workshop', 'https://humem.ai/ontology/location': 'Paris', 'https://humem.ai/ontology/memoryID': '8', 'https://humem.ai/ontology/recalled': '6', 'https://humem.ai/ontology/time': '2024-05-03T14:00:00'})
(rdflib.term.URIRef('https://example.org/entity/Dog'), rdflib.term.URIRef('https://example.org/relationship/is'), rdflib.term.URIRef('https://example.org/entity/Animal'), {'https://humem.ai/ontology/derivedFrom': 'animal_research', 'https://humem.ai/ontology/memoryID': '11', 'https://humem.ai/ontology/recalled': '5', 'https://humem.ai/ontology/strength': '5'})
(rdflib.term.URIRef('https://example.org/person/Alice'), rdflib.term.URIRef('https://example.org/event/met'), rdflib.term.URIRef('https://example.org/person/Bob'), {'https://hume

In [15]:
for memory in memory_system.memory.iterate_memories("episodic"):
    print(memory)

(rdflib.term.URIRef('https://example.org/person/Alice'), rdflib.term.URIRef('https://example.org/relationship/owns'), rdflib.term.URIRef('https://example.org/entity/Dog'), {'https://humem.ai/ontology/emotion': 'curious', 'https://humem.ai/ontology/event': 'Workshop', 'https://humem.ai/ontology/location': 'Paris', 'https://humem.ai/ontology/memoryID': '8', 'https://humem.ai/ontology/recalled': '6', 'https://humem.ai/ontology/time': '2024-05-03T14:00:00'})
(rdflib.term.URIRef('https://example.org/person/Alice'), rdflib.term.URIRef('https://example.org/event/met'), rdflib.term.URIRef('https://example.org/person/Bob'), {'https://humem.ai/ontology/emotion': 'happy', 'https://humem.ai/ontology/event': 'Coffee meeting', 'https://humem.ai/ontology/location': 'New York', 'https://humem.ai/ontology/memoryID': '0', 'https://humem.ai/ontology/recalled': '6', 'https://humem.ai/ontology/time': '2024-04-27T15:00:00'})
(rdflib.term.URIRef('https://example.org/person/Alice'), rdflib.term.URIRef('https:

In [16]:
for memory in memory_system.memory.iterate_memories("semantic"):
    print(memory)

(rdflib.term.URIRef('https://example.org/entity/Dog'), rdflib.term.URIRef('https://example.org/relationship/is'), rdflib.term.URIRef('https://example.org/entity/Animal'), {'https://humem.ai/ontology/derivedFrom': 'animal_research', 'https://humem.ai/ontology/memoryID': '11', 'https://humem.ai/ontology/recalled': '5', 'https://humem.ai/ontology/strength': '5'})
(rdflib.term.URIRef('https://example.org/entity/Dog'), rdflib.term.URIRef('https://example.org/relationship/is'), rdflib.term.URIRef('https://example.org/entity/Animal'), {'https://humem.ai/ontology/derivedFrom': 'animal_research', 'https://humem.ai/ontology/memoryID': '12', 'https://humem.ai/ontology/recalled': '5', 'https://humem.ai/ontology/strength': '5'})
(rdflib.term.URIRef('https://example.org/person/John'), rdflib.term.URIRef('https://example.org/relationship/loves'), rdflib.term.URIRef('https://example.org/entity/Animal'), {'https://humem.ai/ontology/derivedFrom': 'animal_research', 'https://humem.ai/ontology/memoryID': 

In [17]:
len(list(memory_system.memory.iterate_memories()))

19

In [18]:
for memory in memory_system.memory.iterate_memories():
    print(memory)

(rdflib.term.URIRef('https://example.org/person/Alice'), rdflib.term.URIRef('https://example.org/relationship/owns'), rdflib.term.URIRef('https://example.org/entity/Dog'), {'https://humem.ai/ontology/emotion': 'curious', 'https://humem.ai/ontology/event': 'Workshop', 'https://humem.ai/ontology/location': 'Paris', 'https://humem.ai/ontology/memoryID': '8', 'https://humem.ai/ontology/recalled': '6', 'https://humem.ai/ontology/time': '2024-05-03T14:00:00'})
(rdflib.term.URIRef('https://example.org/entity/Dog'), rdflib.term.URIRef('https://example.org/relationship/is'), rdflib.term.URIRef('https://example.org/entity/Animal'), {'https://humem.ai/ontology/derivedFrom': 'animal_research', 'https://humem.ai/ontology/memoryID': '11', 'https://humem.ai/ontology/recalled': '5', 'https://humem.ai/ontology/strength': '5'})
(rdflib.term.URIRef('https://example.org/person/Alice'), rdflib.term.URIRef('https://example.org/event/met'), rdflib.term.URIRef('https://example.org/person/Bob'), {'https://hume

In [19]:
memory

(rdflib.term.URIRef('https://example.org/person/David'),
 rdflib.term.URIRef('https://example.org/event/invited'),
 rdflib.term.URIRef('https://example.org/person/Alice'),
 {'https://humem.ai/ontology/emotion': 'excited',
  'https://humem.ai/ontology/event': 'Conference meeting',
  'https://humem.ai/ontology/location': 'London',
  'https://humem.ai/ontology/memoryID': '4',
  'https://humem.ai/ontology/recalled': '6',
  'https://humem.ai/ontology/time': '2024-05-01T10:00:00'})